Name: Kai Chung, Ying

Course Name: CSC 594

Course Section: 901

Extra Credit

## Import Training set

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import nltk

#Load the Training Dataset
df_train = pd.read_csv("WSJ-train.txt", sep=" ", header=None)
df_train.columns = ["word", "tag", "other"]
#unigrams=df_train.word #Create 
#voca = set(unigrams)

bigrams_lst=[] #Initialize the bigram list
#Combine the Word and its Tag into into Tuples format (i.e. Tuple)
for i in range(0,df_train.shape[0]):
    bigrams_lst += [tuple(df_train.iloc[i][["word", "tag"]])]

#Check the first 10 items
bigrams_lst[0:10]

[('Confidence', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('pound', 'NN'),
 ('is', 'VBZ'),
 ('widely', 'RB'),
 ('expected', 'VBN'),
 ('to', 'TO'),
 ('take', 'VB'),
 ('another', 'DT')]

## Calculate Conditional Frequency Distribution of Bigrams

In [2]:
#Find the Conditional Frequency Distribution of Bigrams
cfd=nltk.ConditionalFreqDist(bigrams_lst)

## Load WSJ Testing set

In [5]:
#Loading the Testing Set 1
df_WSJ_test = pd.read_csv("WSJ-test.txt", sep=" ", header=None)
df_WSJ_test.columns = ["word", "tag", "other"]
df_WSJ_test.head(5)

,word,tag,other
0,Rockwell,NNP,B-NP
1,International,NNP,I-NP
2,Corp.,NNP,I-NP
3,'s,POS,B-NP
4,Tulsa,NNP,I-NP


## Load POS Testing set

In [6]:
#Loading the Testing Set 1
df_POS_test = pd.read_csv("POS-test.txt", sep=" ", header=None)
df_POS_test.columns = ["word"]
df_POS_test.head(5)

,word
0,i
1,'d
2,like
3,to
4,go


## Predict Dataset

In [19]:
#Predict the Tag of the Test dataset
#dataset = df_train
#dataset = df_WSJ_test
dataset = df_POS_test

pred_tag_lst = []
for i in range(0, dataset.shape[0]):# Iterate the Testing dataset
    w=dataset.iloc[i]["word"]
    try:
        #Assign the Tag with the highest probability (i.e.Count) from the Training dataset
        pred_tag_lst.append(cfd[w].max())
    except:
        #Assign Tag = "NN" for Unknown words
        pred_tag_lst.append("NN")

## Convert the String labels to Numeric labels

In [18]:
import numpy as np
str_label = sorted(list(set(df_train.tag)))

#Transfer the actual Tag to a list
actual_tag_lst = list(dataset.tag)

#Initialize lists
actual_tag_num_lst=[]
pred_tag_num_lst=[]

#Convert Actual Tag into Numeric Labels 
for item in actual_tag_lst:
    actual_tag_num_lst.append(str_label.index(item))

#Convert Predicted Tag into Numeric Labels    
pred_tag_num_lst=[]
for item in pred_tag_lst:
    pred_tag_num_lst.append(str_label.index(item))
    

## Adjust Tag by adding rules

In [21]:
pred_tag_lst2 = pred_tag_lst.copy()

#Setup the Special Character list
special_char_lst=["?","#","""'""",'\"',"(",")",",",".",":"],

#Handling Tag =="NN" Cases
for i in range(len(pred_tag_lst)): #Iterate Predicted Tag List
    w = dataset.word.iloc[i]
    pred_tag = pred_tag_lst[i]
    
    #If last character of word == "s" and tag == "NN"
    #Convert the tag to "NNS"
    if w[-1].lower() == "s" and pred_tag =="NN":
        pred_tag_lst2[i]="NNS"
        print("1")
        continue
    #If last 2 characters of word == "ed" and tag == "NN"
    #Convert the tag to "VBN"
    elif w[-2:].lower() == "ed" and pred_tag =="NN":
        pred_tag_lst2[i]="VBN"
        print("2")
        continue
    #If last 3 characters of word == "ing" and tag == "NN"
    #Convert the tag to "VBN"    
    elif w[-3:].lower() == "ing" and pred_tag =="NN":
        pred_tag_lst2[i]="VBG"
        print("3")
        continue


#NN -> VBN if the tag of the preceding word is "VBP
for i in range(1,len(pred_tag_lst)): #Iterate Predicted Tag List
    w = dataset.word.iloc[i]
    pred_tag = pred_tag_lst[i]
    
    if (pred_tag_lst[i-1] == "VBP"
        and pred_tag =="NN"):
        pred_tag_lst2[i]="VBN"
        print("1")
        continue
        
#NN -> NNP if the tag of i+1 and i+2 words are "NNP"
for i in range(0,len(pred_tag_lst)-2): #Iterate Predicted Tag List
    w = dataset.word.iloc[i]
    pred_tag = pred_tag_lst[i]
    
    if (pred_tag_lst[i+1] == "NNP" or pred_tag_lst[i+2] == "NNP"
        and pred_tag =="NN"):
        pred_tag_lst2[i]="NNP"
        print("1")
        continue
        
#NN -> VB if the tag of preceding word is "TO"
for i in range(1,len(pred_tag_lst)): #Iterate Predicted Tag List
    w = dataset.word.iloc[i]
    pred_tag = pred_tag_lst[i]
    
    if (pred_tag_lst[i-1] == "TO"
        and pred_tag =="NN"):
        pred_tag_lst2[i]="VB"
        print("1")
        continue
        
#If any one of the previous three tags is "MD" AND NOT a Special Character AND Tag = "VBP"
#Then Convert the tag to "VB"  
for i in range(len(pred_tag_lst)):#Iterate Predicted Tag List
    if i==0: continue
    elif i==1:
        if pred_tag_lst2[i-1] == "MD" and pred_tag_lst2[i] == "VBP":
            pred_tag_lst2[i] == "VB"
            print("5")
    elif i==2:
        if ((pred_tag_lst2[i-2] == "MD" or pred_tag_lst2[i-1] == "MD") 
            and pred_tag_lst2[i-1] not in special_char_lst
            and pred_tag_lst2[i] == "VBP"):
            
            pred_tag_lst2[i] == "VB"
            print("6")
    else:
        if ((pred_tag_lst2[i-3] == "MD" or pred_tag_lst2[i-2] == "MD" or pred_tag_lst2[i-1] == "MD") 
            and (pred_tag_lst2[i-2] not in special_char_lst  and pred_tag_lst2[i-1] not in special_char_lst)
            and pred_tag_lst2[i] == "VBP"):
            
            pred_tag_lst2[i] == "VB"
            print("7")
            
#If any one of the previous two tags is "MD" AND NOT a Special Character AND Tag = "NN"
#Then Convert the tag to "VB"  
for i in range(len(pred_tag_lst)):#Iterate Predicted Tag List
    if i==0: continue
    elif i==1:
        if pred_tag_lst2[i-1] == "MD" and pred_tag_lst2[i] == "NN":
            pred_tag_lst2[i] == "VB"
            print("8")
    else:
        if ((pred_tag_lst2[i-2] == "MD" or pred_tag_lst2[i-1] == "MD") 
            and pred_tag_lst2[i-1] not in special_char_lst
            and pred_tag_lst2[i] == "NN"):
            pred_tag_lst2[i] == "VB"
            print("9")

#Any one of the previous two tags is "DT" AND NOT a Special Character AND Tag = "VB"
#Then Convert the tag to "NN" 
for i in range(len(pred_tag_lst)):#Iterate Predicted Tag List
    if i==0: continue
    elif i==1:
        if pred_tag_lst2[i-1] == "DT" and pred_tag_lst2[i] == "VB":
            pred_tag_lst2[i] == "NN"
            print("10")
    else:
        if ((pred_tag_lst2[i-2] == "DT" or pred_tag_lst2[i-1] == "DT") 
            and pred_tag_lst2[i-1] not in special_char_lst
            and pred_tag_lst2[i] == "VB"):
            pred_tag_lst2[i] == "NN"
            print("11")

3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
1
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
3
3
3
3
3
3
1
1
1
1
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
3
3
3
1
1
1
1
1
3
3
1
1
1
1
1
1
1
1
3
1
3
1
1
1
1
1
1
1
1
1
2
2
2
2
3
3
1
1
1
3
3
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
1
1
1
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7


## Putting the word / tag into a dataframe and write to CSV

In [33]:
pred_df = pd.DataFrame(list(zip(list(df_POS_test.word),pred_tag_lst2)))

with open("Output_held-out.txt", "w") as f:
    for i in range(0,pred_df.shape[0]):
        f.write("%s %s\n"%(pred_df.iloc[i][0],pred_df.iloc[i][1]))

## Convert the String labels to Numeric labels

In [16]:
pred_tag_num_lst2=[]
for item in pred_tag_lst2:
    pred_tag_num_lst2.append(str_label.index(item))
    

## Measuring Metrics

In [17]:
from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf, linewidth=400)

#Confusion Matrix (Before Transformation Ruling)
test1_cm = confusion_matrix(actual_tag_num_lst,pred_tag_num_lst,labels=list(np.arange(0,44))) 
#Confusion Matrix (After Transformation Ruling)
test1_rule_cm = confusion_matrix(actual_tag_num_lst,pred_tag_num_lst2,labels=list(np.arange(0,44)))

pos_cnt=0
pos_rule_cnt=0

#with open("result-WSJ-test.txt", "w") as f:
#Recall / Precision / f1 score /Accuracy
for i in range(0,test1_cm.shape[0]):
    print("Tag:{0:7}Before  After".format(str_label[i]))
    print("{0}".format("-"*25))
    
#####Recall
    if sum(test1_cm[i]) ==0:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i], sum(test1_cm[i]),0)) #Recall
        var_recall=0
        print("Recall:  {0:0.3f}".format(var_recall), end=",  ") #Recall
    else:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i], sum(test1_cm[i]),(test1_cm[i,i] / sum(test1_cm[i])))) #Recall
        var_recall =(test1_cm[i,i] / sum(test1_cm[i]))
        print("Recall:    {0:0.3f}".format(var_recall), end=",  ") #Recall
        
        
    if sum(test1_rule_cm[i]) ==0:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i], sum(test1_rule_cm[i]),0)) #Recall
        var_recall=0
        print("%0.3f"%(var_recall)) #Recall
    else:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i], sum(test1_rule_cm[i]),(test1_rule_cm[i,i] / sum(test1_rule_cm[i])))) #Recall
        var_recall =(test1_rule_cm[i,i] / sum(test1_rule_cm[i]))
        print("%0.3f"%(var_recall)) #Recall
        
    
#####Precision
    if sum(test1_cm[:,i]) ==0:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i],sum(test1_cm[:,i]) ,0)) #Precision
        var_precision=0
        print("Precision: {0:0.3f}".format(var_precision), end=",  ") #Precision
    else:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i],sum(test1_cm[:,i]) ,(test1_cm[i,i] / sum(test1_cm[:,i])))) #Precision
        var_precision =(test1_cm[i,i] / sum(test1_cm[:,i]))
        print("Precision: {0:0.3f}".format(var_precision), end=",  ") #Precision
    
    if sum(test1_rule_cm[:,i]) ==0:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i],sum(test1_rule_cm[:,i]) ,0)) #Precision
        var_precision=0
        print("%0.3f"%(var_precision)) #Precision
    else:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i],sum(test1_rule_cm[:,i]) ,(test1_rule_cm[i,i] / sum(test1_rule_cm[:,i])))) #Precision
        var_precision =(test1_rule_cm[i,i] / sum(test1_rule_cm[:,i]))
        print("%0.3f"%(var_precision)) #Precision
    
##### F1 Score    
    if sum(test1_cm[i]) ==0 or sum(test1_cm[:,i]) ==0:
        var_f1=0
        print("f1 score:  {0:0.3f}".format(var_f1), end=",  ") #f1 score
    else:
        var_recall=(test1_cm[i,i] / sum(test1_cm[i]))
        var_precision=(test1_cm[i,i] / sum(test1_cm[:,i]))
        var_f1 = 2/((1/var_recall)+(1/var_precision))
        print("f1 score:  {0:0.3f}".format(var_f1), end=",  ") #f1 score

        
    if sum(test1_rule_cm[i]) ==0 or sum(test1_rule_cm[:,i]) ==0:
        var_f1=0
        print("%0.3f"%(var_f1)) #f1 score
    else:
        var_recall = (test1_rule_cm[i,i] / sum(test1_rule_cm[i]))
        var_precision = (test1_rule_cm[i,i] / sum(test1_rule_cm[:,i]))
        var_f1 = 2/((1/var_recall)+(1/var_precision))
        print("%0.3f"%(var_f1)) #f1 score
    print()
    
    pos_cnt += test1_cm[i,i]
    pos_rule_cnt += test1_rule_cm[i,i]
    
print(pos_cnt)
print(pos_rule_cnt)
print("Accuracy: ",round((pos_cnt / len(actual_tag_num_lst)),3), end=",  ")
print(round((pos_rule_cnt / len(actual_tag_num_lst)),3))

Tag:#      Before  After
-------------------------
Recall:    1.000,  1.000
Precision: 1.000,  1.000
f1 score:  1.000,  1.000

Tag:$      Before  After
-------------------------
Recall:    1.000,  1.000
Precision: 1.000,  1.000
f1 score:  1.000,  1.000

Tag:''     Before  After
-------------------------
Recall:    0.984,  0.826
Precision: 1.000,  1.000
f1 score:  0.992,  0.905

Tag:(      Before  After
-------------------------
Recall:    1.000,  0.675
Precision: 1.000,  1.000
f1 score:  1.000,  0.806

Tag:)      Before  After
-------------------------
Recall:    1.000,  0.922
Precision: 1.000,  1.000
f1 score:  1.000,  0.959

Tag:,      Before  After
-------------------------
Recall:    1.000,  0.898
Precision: 1.000,  1.000
f1 score:  1.000,  0.946

Tag:.      Before  After
-------------------------
Recall:    1.000,  0.828
Precision: 1.000,  1.000
f1 score:  1.000,  0.906

Tag::      Before  After
-------------------------
Recall:    1.000,  0.899
Precision: 1.000,  1.000
f1 score: 

# End!!!

In [ ]:
from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf, linewidth=400)
test1_cm = confusion_matrix(actual_tag_num_lst
                      ,pred_tag_num_lst
                      ,labels=list(np.arange(0,44))) 

pos_cnt=0
#Recall and Precision
for i in range(0,test1_cm.shape[0]):
    print("Tag is ",str_label[i])
    if sum(test1_cm[i]) ==0:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i], sum(test1_cm[i]),0)) #Recall
        var_recall=0
        print("Recall: %s, %0.3f"%(str_label[i],var_recall)) #Recall
    else:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i], sum(test1_cm[i]),(test1_cm[i,i] / sum(test1_cm[i])))) #Recall
        var_recall =(test1_cm[i,i] / sum(test1_cm[i]))
        print("Recall: %s, %0.3f"%(str_label[i],var_recall)) #Recall
    
    if sum(test1_cm[:,i]) ==0:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i],sum(test1_cm[:,i]) ,0)) #Precision
        var_precision=0
        print("Precision: %s, %0.3f"%(str_label[i],var_precision)) #Precision
    else:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_cm[i,i],sum(test1_cm[:,i]) ,(test1_cm[i,i] / sum(test1_cm[:,i])))) #Precision
        var_precision =(test1_cm[i,i] / sum(test1_cm[:,i]))
        print("Precision: %s, %0.3f"%(str_label[i],var_precision)) #Precision
    
    if var_recall == 0 or var_precision==0:
        var_f1=0
        print("f1 score: %s, %0.3f"%(str_label[i],var_f1)) #f1 score
    else:
        var_f1 = 2/((1/var_recall)+(1/var_precision))
        print("f1 score: %s, %0.3f"%(str_label[i],var_f1)) #f1 score
    print()
    pos_cnt += test1_cm[i,i]
print("The overall accuracy is",pos_cnt / len(actual_tag_num_lst))

In [44]:
from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf, linewidth=400)
test1_rule_cm = confusion_matrix(actual_tag_num_lst
                      ,pred_tag_num_lst2
                      ,labels=list(np.arange(0,44))) 

pos_cnt=0
#Recall and Precision
for i in range(0,test1_rule_cm.shape[0]):
    print("Tag is ",str_label[i])
    if sum(test1_rule_cm[i]) ==0:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i], sum(test1_rule_cm[i]),0)) #Recall
        var_recall=0
        print("Recall: %s, %0.3f"%(str_label[i],var_recall)) #Recall
    else:
        #print("Recall: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i], sum(test1_rule_cm[i]),(test1_rule_cm[i,i] / sum(test1_rule_cm[i])))) #Recall
        var_recall =(test1_rule_cm[i,i] / sum(test1_rule_cm[i]))
        print("Recall: %s, %0.3f"%(str_label[i],var_recall)) #Recall
    
    if sum(test1_rule_cm[:,i]) ==0:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i],sum(test1_rule_cm[:,i]) ,0)) #Precision
        var_precision=0
        print("Precision: %s, %0.3f"%(str_label[i],var_precision)) #Precision
    else:
        #print("Precision: %s,%d ,%d, %0.3f"%(str_label[i],test1_rule_cm[i,i],sum(test1_rule_cm[:,i]) ,(test1_rule_cm[i,i] / sum(test1_rule_cm[:,i])))) #Precision
        var_precision =(test1_rule_cm[i,i] / sum(test1_rule_cm[:,i]))
        print("Precision: %s, %0.3f"%(str_label[i],var_precision)) #Precision
    
    
    
    
    if var_recall == 0 or var_precision==0:
        var_f1=0
        print("f1 score: %s, %0.3f"%(str_label[i],var_f1)) #f1 score
    else:
        var_f1 = 2/((1/var_recall)+(1/var_precision))
        print("f1 score: %s, %0.3f"%(str_label[i],var_f1)) #f1 score
    print()
    pos_cnt += test1_rule_cm[i,i]
print("The overall accuracy is",pos_cnt / len(actual_tag_num_lst))

Tag is  #
Recall: #, 1.000
Precision: #, 1.000
f1 score: #, 1.000

Tag is  $
Recall: $, 1.000
Precision: $, 1.000
f1 score: $, 1.000

Tag is  ''
Recall: '', 0.984
Precision: '', 1.000
f1 score: '', 0.992

Tag is  (
Recall: (, 1.000
Precision: (, 1.000
f1 score: (, 1.000

Tag is  )
Recall: ), 1.000
Precision: ), 1.000
f1 score: ), 1.000

Tag is  ,
Recall: ,, 1.000
Precision: ,, 1.000
f1 score: ,, 1.000

Tag is  .
Recall: ., 1.000
Precision: ., 1.000
f1 score: ., 1.000

Tag is  :
Recall: :, 1.000
Precision: :, 1.000
f1 score: :, 1.000

Tag is  CC
Recall: CC, 1.000
Precision: CC, 1.000
f1 score: CC, 1.000

Tag is  CD
Recall: CD, 0.840
Precision: CD, 0.999
f1 score: CD, 0.913

Tag is  DT
Recall: DT, 0.990
Precision: DT, 0.996
f1 score: DT, 0.993

Tag is  EX
Recall: EX, 1.000
Precision: EX, 0.960
f1 score: EX, 0.980

Tag is  FW
Recall: FW, 0.750
Precision: FW, 1.000
f1 score: FW, 0.857

Tag is  IN
Recall: IN, 0.998
Precision: IN, 0.958
f1 score: IN, 0.978

Tag is  JJ
Recall: JJ, 0.833
Preci